# Priority Queues

In diesem Notebook werden wir zwei einfache Implementationen des ADTs *Priority Queue* anschauen. Diese Implementationen dienen nur zur Illustration des Prinzips, und zur Veranschaulichung, dass auch hier verschiedene Implementation verschiedene Kompromisse machen. Für eine praktische Implementation werden wir später einen Heap verwenden.



## Zwei einfache Implementationen mittels Arrays

#### Implementation mit unsortiertem Array

Die erste Implementation verwendet ein einfaches, unsortiertes (dynamisches) Array. Bei dieser Implementation ist die ```insert``` Methode sehr effizient. Die Hauptarbeit passiert, wenn die Methode ```max``` oder ```delmax``` aufgerufen wird. In diesem Fall wird das grösste Element gesucht und mit dem letzten Element vertauscht. Damit kann es einfach zurückgegeben werden.

In [ ]:
class MaxPQUnorderedArray:
    
    def __init__(self):
        self._data = []
    
    def insert(self, k):
        self._data.append(k)
    
    def max(self):
        self._largestToEnd()
        return self._data[-1] #[-1] greift auf letztes Element zu
            
    def delMax(self):
        self._largestToEnd()
        return self._data.pop()
    
    def _largestToEnd(self):
        if len(self._data) == 0:
            return
        
        maxElem = self._data[0]            
        maxIndex = 0
        for i, d in enumerate(self._data):
            if (maxElem < d):
                maxElem = self._data[i]
                maxIndex = i
        self._data[maxIndex], self._data[-1] = self._data[-1], self._data[maxIndex]
        
    
        
    def isEmpty(self):
        return len(self._data) == 0
    
    def size(self):
        return len(self._data)

#### Implementation mit sortiertem Array

In dieser zweiten Implementation, passiert die Hauptarbeit in der Methode ```insert```. Unsere Implementation stellt sicher, dass das neue Element jeweils an die richtige Position im Array eingefügt wird. Im Vergleich zur vorherigen Methode, sind dafür die Methoden ```max``` und ```delMax``` sehr effizient.

In [ ]:
class MaxPQOrderedArray:
    
    def __init__(self):
        self._data = []
    
    def insert(self, k):
        
        # Suche im sortierten Array data die Position vom neuen Element
        idx = 0
        while (idx < len(self._data) and self._data[idx] < k):            
            idx += 1
        self._data.insert(idx, k)        
        
    def max(self):
        if self.isEmpty():
            return None
        else:
            return self._data[-1]
            
    def delMax(self):
        return self._data.pop()
        
        
    def isEmpty(self):
        return len(self._data) == 0
    
    def size(self):
        return len(self._data)

#### Komplexität

Nun schauen wir uns die Laufzeit der beiden Implementationen an. Wir nutzen dafür wieder das Python Modul ```timeit``` welches wir schon früher kennengelernt haben. 

In [ ]:
import timeit
import random

Die folgenden beiden Hilfsfunktionen fügen jeweils $N$-Elemente hinzu oder entfernen diese aus der Queue.

In [ ]:
def insertNElements(pq, N):
    for i in range(0, N):
        pq.insert(random.randint(0, N))


In [ ]:
def removeLargestNElements(pq, N):
    for i in range(0, N):
        pq.delMax()

Wir können die Tests nun ausführen. 

In [ ]:
orderedPQ = MaxPQOrderedArray()
unorderedPQ = MaxPQUnorderedArray()

print("insert ordered ", timeit.timeit(lambda: insertNElements(orderedPQ, 10000), number = 1))
print("insert unordered", timeit.timeit(lambda: insertNElements(unorderedPQ, 10000), number = 1))

# ACHTUNG: Wir nutzen hier aus, dass orderedPQ und unorderedPQ bereits wegen dem vorigen Experiment gefüllt sind
print("removeLargest ordered", timeit.timeit(lambda: removeLargestNElements(orderedPQ, 10000), number = 1))
print("removeLargest unordered", timeit.timeit(lambda: removeLargestNElements(unorderedPQ, 10000), number = 1))

Wie erwartet ist das Einfügen sehr viel teurer, wenn wir für die Implementation ein geordnetes Array nutzen. Dafür wird das entfernen des grössten Elements entsprechend effizienter. 

### Beispielanwendung von Priority-Queues

Eine typische Beispielanwendung ist, dass man aus einem sehr grossen Datenstrom die extremsten Elemente extrahieren möchte. 

In unserem Beispiel besteht der Datenstrom aus normalverteilten Zufallswerden. Wir schreiben einen Client, der die $M$ kleinsten Werte die generiert wurden ausgibt. 

Die folgende Funktion generiert einen Stream von $N$ normalverteilen Zufallszahlen. Das Python Keyword yield stellt hier sicher, dass die Zahlen jeweils einzeln generiert werden, so dass wir nicht eine grosse Liste von Zahlen speichern müssen. 

In [ ]:
import random

def numberGen(N):
    num = 0
    while num < N:
        yield random.gauss(0, 1)
        num += 1

Da wir hier nur eine MaxPQ zur Verfügung haben, ist es etwas einfacher die kleinsten Elemente zu suchen. Wir speichern jedes Element in der PriorityQueue, lassen aber nur jeweils die $M$ grössten Elemente in der Queue. Die anderen werden sofort wieder entfernt.

In [ ]:
def printSmallestNumbers(M, N):
    pq = MaxPQUnorderedArray()
    for number in numberGen(N):

        pq.insert(number)
        if pq.size() > M:
            pq.delMax()
        
    while not pq.isEmpty():
        print(pq.delMax())


In unserer Anwendung können wir nun mit diesem Algorithmus ermitteln, wie extrem denn Werte die zufällig von einer Standard-Normalverteilung gezogen werden eigentlich werden können. 

In [ ]:
printSmallestNumbers(5, 1000000)

Wir sehen, dass auch wenn wir 1 Million zufällige Elemente ziehen, kaum eines extremer als -5 ist. 

*Anmerkung für Statistik interessiert:  Unter der Normalverteilung sind wie wir sehen extreme Werte sehr unwahrscheinlich. Deshalb ist die Normalverteilung kein gutes Modell für Prozesse, bei denen auch ab und zu mal ein grösserer Wert vorkommt.*